In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!

**HelperFunction**

In [2]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

def get_noise(n_samples,z_dim,device = 'cpu'):
    return torch.randn(n_samples,z_dim,device = device)

**Generator**

In [3]:
def make_gen_block(input_chan,output_chan,kernel_size=3,stride=2,final_layer=False):
    if not final_layer:
       return nn.Sequential(
         nn.ConvTranspose2d(input_chan,output_chan,kernel_size,stride = stride),
         nn.BatchNorm2d(output_chan),
         nn.ReLU(inplace = True)
       )
    else:
       return nn.Sequential(
         nn.ConvTranspose2d(input_chan,output_chan,kernel_size,stride = stride),
         nn.Tanh()
       )

class Generator(nn.Module):
    def __init__(self,z_dim = 10,im_chan = 1,hidden_dim = 64):
        super(Generator,self).__init__()
        self.z_dim = z_dim
        self.gen = nn.Sequential(
            make_gen_block(z_dim,hidden_dim * 4),
            make_gen_block(hidden_dim * 4,hidden_dim * 2,kernel_size = 4,stride = 1),
            make_gen_block(hidden_dim * 2,hidden_dim),
            make_gen_block(hidden_dim,im_chan,kernel_size=4,final_layer=True),
        )

    def unsqueeze_noise(self, noise):
        return noise.view(len(noise),self.z_dim,1,1)
    
    def forward(self,noise):
        x = self.unsqueeze_noise(noise)
        return self.gen(x)

    def get_gen(self):
        return self.gen

**Discriminator**

In [4]:
def make_disc_block(input_chan,output_chan,kernel_size=4,stride=2,final_layer=False):
    if not final_layer:
        return nn.Sequential(
            nn.Conv2d(input_chan,output_chan,kernel_size,stride=stride),
            nn.BatchNorm2d(output_chan),
            nn.LeakyReLU(0.2)
        )
    else:
        return nn.Sequential(
            nn.Conv2d(input_chan,output_chan,kernel_size,stride=stride)
        )

class Discriminator(nn.Module):
    def __init__(self,im_chan=1,hidden_chan=16):
        super(Discriminator,self).__init__()
        self.disc = nn.Sequential(
            make_disc_block(im_chan,hidden_chan),
            make_disc_block(hidden_chan,hidden_chan * 2),
            #make_disc_block(hidden_chan * 2,hidden_chan * 4),
            make_disc_block(hidden_chan * 2,1,final_layer=True),
        )
    
    def forward(self,img):
        disc_pred = self.disc(img)
        return disc_pred.view(len(disc_pred), -1)

**Initialize Component**

In [5]:
#set hyperparameter
z_dim = 64
batch_size = 128
n_epochs = 200
display_step = 500
lr = 0.00001
criterion = nn.BCEWithLogitsLoss()
beta_1 = 0.5 
beta_2 = 0.999

#convert data to tensor
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size = batch_size,
    shuffle = True
)

#set device
device = 'cuda'

In [6]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta_1, beta_2))

# You initialize the weights to the normal distribution
# with mean 0 and standard deviation 0.02
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
gen = gen.apply(weights_init)
disc = disc.apply(weights_init)

**Loss method**

In [7]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    fake_noise = get_noise(num_images,z_dim,device)
    fake_img = gen(fake_noise)
    disc_fake_pred = disc(fake_img.detach()) #to unaffect parameter of generator
    disc_fake_loss = criterion(disc_fake_pred,torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred,torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2

    return disc_loss

def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    fake_noise = get_noise(num_images,z_dim,device)
    fake_image = gen(fake_noise)
    disc_fake_pred = disc(fake_image)
    gen_loss = criterion(disc_fake_pred,torch.ones_like(disc_fake_pred))

    return gen_loss  

**Training**

In [8]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
losses_gen = []
losses_disc = []

In [9]:
for epoch in range(n_epochs):
    for real,_ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        #real = real.view(cur_batch_size, -1).to(device)
        real = real.to(device)
         ### Update discriminator ###

        disc_opt.zero_grad() #zero_out the gradient before backpropagation

        disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)
        disc_loss.backward(retain_graph = True)

        disc_opt.step()

        ### Update discriminator ###

        ### Update generator ###

        gen_opt.zero_grad()

        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward(retain_graph = True)

        gen_opt.step()
         
        ### Update discriminator ###

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        losses_gen.append(mean_generator_loss)
        losses_disc.append(mean_discriminator_loss)

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

Output hidden; open in https://colab.research.google.com to view.